# All imports and csv reads

In [1]:
import pandas as pd
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import warnings
import numpy as np
import math
import pickle

In [3]:
df_anime = pd.read_csv('new_anime.csv')         # list of all animes
df_anime_soup = pd.read_csv('anime_bag_of_words_all.csv')   # bag of words for all anime
df_user = pd.read_csv('user_list_more_than_20.csv')          # list of users with their watch lists
# df_user = pd.read_csv('user_list.csv')
df_user_soup = pd.read_csv('users_bag_of_words_all.csv')    # bag of words for users watch list
# df_matrix = pd.read_csv('anime_matrix_empty.csv')
# df_true = pd.read_csv('anime_matrix_full.csv')

In [20]:

count = CountVectorizer()
list_of_bag_of_words = [df_user_soup.loc[1]['bag of words']]

count_matrix = count.fit_transform(df_anime_soup['bag of words'])
count_matrix2 = count.transform(list_of_bag_of_words)

cosine_sim2 = cosine_similarity(count_matrix, count_matrix2)
sim_scores = list(enumerate(cosine_sim2))

print(sim_scores[1][1][0])

0.2558943964199104


# Data file manipulations and changes go here

removes users/rows with empty watch lists

In [18]:
df_user = df_user.dropna()                      
df_user.to_csv('user_list.csv', index=None)

drop users/row that have a watched count of less than 20; creates new csv file

In [52]:
data = range(len(df_user))                      # used with for loops

for i in tqdm(data):
    watched_count = df_user.loc[i]['watched count']
    if watched_count < 20:
        df_user = df_user.drop(labels=i,axis=0)

df_user

100%|██████████| 9486/9486 [00:00<00:00, 10698.96it/s]


,user,watched count,watched list
0,ilampan,380,19367 18411 17895 18247 18295 18679 18153 1970...
1,Zephyyr,538,31646 9624 857 6547 35427 7817 34618 35712 380...
2,Alister-Dark,34,51096 35180 4181 2904 42938 5114 820 918 15335...
3,AMechaDegenarate,203,50709 41084 48895 47194 41457 11759 22199 47 4...
4,Matthew_Myers,211,49438 48675 50709 49470 30014 21 41389 51096 3...
...,...,...,...
9481,Darkidda,590,38670 32827 36649 14751 1239 35120 38691 28121...
9482,chidoriP,324,48569 51417 48413 42963 50709 40356 51096 4145...
9483,snepback,754,48895 31374 31646 35180 38101 39783 41457 4856...
9484,Prism8Azure,212,49123 3654 38935 32924 8676 9117 11235 12669 6...


In [53]:
df_user.to_csv('user_list_more_than_20.csv', index=None)

# Create csv files that have bag of words

function to make the bag of words for anime list

In [3]:
def create_bag(x):
    result = x['Anime Title']

    if x['Studio'] != 'n/a':
        result += ' ' + x['Studio']
    if x['Source Material'] != 'n/a':
        result += ' ' + x['Source Material']
    if x['Genre'] != 'n/a':
        result += ' ' + x['Genre']
    if x['VAs'] != 'n/a':
        result += ' ' + x['VAs']
    
    punc = '[],\'\"'
    
    for ele in result:
        if ele in punc:
            result = result.replace(ele, "")

    return result

make csv file for anime list with bag of words

In [24]:
df_anime['soup'] = df_anime.apply(create_bag,axis = 1)
df_anime = df_anime.drop(['Rating', 'Studio', 'Source Material', 'Demographic', 'Genre', 'VAs'], axis=1)
df_anime.to_csv("anime_bag_of_words.csv", index=None)

function to make the bag of words for user list

In [82]:
watch_list = (df_user.loc[0]['watched list']).split()
temp = '\'' + watch_list[20] + '\''
index = df_anime_soup.loc[df_anime_soup['Id'] == temp].index[0]
index

2889

my way

In [3]:
data = range(len(df_user))
bag_of_words = []
for i in tqdm(data):
    watch_list = (df_user.loc[i]['watched list']).split()
    bag = ''
    for j in range(len(watch_list)):
        temp = '\'' + watch_list[j] + '\''
        try:
            index = df_anime_soup.loc[df_anime_soup['Id'] == temp].index[0]
            bag += ' ' + df_anime_soup.loc[index]['soup']
        except:
            pass

    bag_of_words.append(bag)

df_user['bag of words'] = bag_of_words
df_user = df_user.drop(['watched count', 'watched list'], axis=1)
df_user.to_csv('user_bag_of_words_full.csv', index=None)

100%|██████████| 9028/9028 [22:23<00:00,  6.72it/s]  


In [4]:
df_user

,user,bag of words
0,ilampan,GalileiDonna A-1Pictures Original Action Adve...
1,Zephyyr,3-gatsunoLion Shaft Manga Drama SliceofLife C...
2,Alister-Dark,3-gatsunoLion2ndSeason Shaft Manga Drama Slic...
3,AMechaDegenarate,86 A-1Pictures Lightnovel Action Drama Sci-Fi...
4,Matthew_Myers,OnePiece ToeiAnimation Manga Action Adventure...
...,...,...
9023,Darkidda,Actors:SongsConnection Drive Music Music Scho...
9024,chidoriP,86Part2 A-1Pictures Lightnovel Action Drama S...
9025,snepback,Shingeki!KyojinChuugakkou ProductionI.G Manga...
9026,Prism8Azure,MacademiWasshoi! Zexcs Lightnovel Comedy Fant...


In [55]:
data = range(len(df_user))
result = []
for i in tqdm(data):
    try:
        user = df_user.loc[i]['user']
        watch_list = (df_user.loc[i]['watched list']).split()
        bag = ''
        for j in range(len(watch_list)):
            try:
                temp = '\'' + watch_list[j] + '\''
                index = df_anime.loc[df_anime['Id'] == temp].index[0]
                bag += ' ' + create_bag(df_anime.loc[index])
            except:
                pass
    except:
        pass
    result.append([user,bag])

outputdf = pd.DataFrame(result, columns=['user','bag of words'])
outputdf.to_csv('user_bag_of_words_full.csv', index=None)

100%|██████████| 9028/9028 [26:47<00:00,  5.62it/s]  


# create csv file that have 0s or 1s based on similarity threshold

In [13]:
df_anime_soup['bag of words']

0      KonoSubarashiiSekainiShukufukuwo!2 StudioDeen ...
1      Kobayashi-sanChinoMaidDragon KyotoAnimation Ma...
2      YoujoSenki Nut Lightnovel Action Isekai Milita...
3      Masamune-kunnoRevenge SILVERLINK. Manga Comedy...
4      KuzunoHonkai Lerche Manga Drama Romance School...
                             ...                        
551    EgaonoDaika TatsunokoProduction Original Drama...
552    Nippon-ichinoOtokonoTamashii StudioDeen Unknow...
553    LupintheThird:MineFujikotoIuOnna TMSEntertainm...
554    BakumatsuGijindenRoman TMSEntertainment Other ...
555    Hakuouki:Reimeiroku StudioDeen Visualnovel Act...
Name: bag of words, Length: 556, dtype: object

In [55]:
df_user_soup = pd.read_csv('users_bag_of_words_all.csv')
df_matrix = pd.read_csv('anime_matrix_empty.csv')
df_anime_soup = pd.read_csv('anime_bag_of_words_all.csv')
df_anime = pd.read_csv('top_and_bottom_anime.csv')
df_sim_scores = pd.read_csv('user_sim_scores.csv')
df_sim_scores = df_sim_scores.drop(columns='anime')

In [57]:
sim_scores_list = df_sim_scores.values
sim_scores_list

array([[0.15637401, 0.17370149, 0.04436237, ..., 0.22015965, 0.14409153,
        0.14940909],
       [0.12688825, 0.2558944 , 0.09719308, ..., 0.24117281, 0.15784436,
        0.15474203],
       [0.13686659, 0.13725817, 0.08339245, ..., 0.17479651, 0.06771587,
        0.16632715],
       ...,
       [0.20322706, 0.26805249, 0.18048302, ..., 0.2738193 , 0.08243708,
        0.25971905],
       [0.06799001, 0.08036035, 0.02479932, ..., 0.08695374, 0.05034351,
        0.08200363],
       [0.19144578, 0.15230255, 0.14664245, ..., 0.1630746 , 0.10075643,
        0.20998562]])

In [3]:
def similarity_threshold(idx, df_user_soup = df_user_soup, df_anime_soup = df_anime_soup):
    # df_anime[df_user_soup.loc[idx]['user']] = '0'

    count = CountVectorizer()
    list_of_bag_of_words = [df_user_soup.loc[idx]['bag of words']]
    sim_scores_list = df_sim_scores.values

    try:
        count_matrix = count.fit_transform(df_anime_soup['bag of words'])
        count_matrix2 = count.transform(list_of_bag_of_words)

        cosine_sim2 = cosine_similarity(count_matrix, count_matrix2)
        sim_scores = list(enumerate(cosine_sim2))

        for i in range(len(df_anime)):
            try:
                df_matrix.at[i, df_user_soup.loc[idx]['user']] = sim_scores[i][1][0]
                # if sim_scores[i][1][0] > 0.22:
                #     df_matrix.at[i, df_user_soup.loc[idx]['user']] = 1
            except:
                pass
    except:
        pass

In [4]:
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
data = range(len(df_user_soup))
# percentages = [0.2,0.22,0.24,0.26,0.28,0.3,0.32,0.34,0.36,0.38,0.4,0.42,0.44,0.46,0.48,0.5]

# for i in percentages:
#     print('current percentage is ' + str(i))
for j in tqdm(data):
    similarity_threshold(j,df_user_soup, df_anime_soup)
    # filename = 'user_threshold_full_' + str(i) + '.csv'
# df_matrix.to_csv('user_threshold_all.csv', index=None)
    
# df_anime = df_anime.drop(['Anime Title', 'Rating', 'Studio', 'Source Material', 'Demographic', 'Genre', 'VAs'],axis = 1)
df_matrix.to_csv('user_sim_scores.csv', index=None)

100%|██████████| 9028/9028 [10:44<00:00, 14.00it/s]


# create csv based on true value of whether or not a user watch a show

creates empty csv file filled with 0s

In [23]:
df_user = pd.read_csv('user_list_more_than_20.csv')
df_anime = pd.read_csv('top_and_bottom_anime.csv')

In [24]:
header = ['anime']
for i in range(len(df_user)):
    header.append(df_user.loc[i]['user'])

result = []
for i in range(len(df_anime)):
    row = [df_anime.loc[i]['Id'].replace('\'','')]
    for j in range(len(df_user)):
        row.append(0)
    result.append(row)

outputdf = pd.DataFrame(result, columns=header)
outputdf.to_csv('anime_matrix_empty.csv', index=False)

In [25]:
df_matrix = pd.read_csv('anime_matrix_empty.csv')
df_user = pd.read_csv('user_list_more_than_20.csv')
df_anime = pd.read_csv('top_and_bottom_anime.csv')

In [26]:
users = []
for i in range(len(df_user)):
    users.append(df_user.loc[i]['user'])
    
for i in tqdm(range(len(users))):
    try:
        watch_list = df_user.loc[i]["watched list"].split()
        for j in range(len(df_anime)):
            if df_anime.loc[j]['Id'].replace('\'','') in watch_list:
                df_matrix.at[j,users[i]] = 1
    except:
        pass

100%|██████████| 9028/9028 [07:29<00:00, 20.08it/s]


In [27]:
df_matrix.to_csv('anime_matrix_all.csv', index=None)

# ml models

knn model fit

In [43]:
def model_config(x,y = df_true):
    x = x
    y = y
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
    knn = KNeighborsClassifier()
    knn.fit(x_train,y_train)
    result = knn.score(x_test,y_test)
    return result

In [104]:
df_sim_scores = pd.read_csv('user_sim_scores.csv')
df_sim_scores

,anime,ilampan,Zephyyr,Alister-Dark,AMechaDegenarate,Matthew_Myers,eezairi,Neji4460,Aguena_Jun,andrzejszczelba,...,Jaxelar,BitSneakySvenpai,PeelyPeelyBoi,Tray1990,Kilax,Darkidda,chidoriP,snepback,Prism8Azure,urban_orange
0,32937,0.217252,0.223515,0.089722,0.150298,0.182979,0.273474,0.258708,0.268929,0.240361,...,0.258890,0.257032,0.238646,0.225111,0.251968,0.210780,0.220746,0.233775,0.224448,0.143757
1,33206,0.146575,0.187004,0.106148,0.116709,0.143739,0.158145,0.173802,0.214696,0.174723,...,0.150553,0.173099,0.192837,0.150311,0.166464,0.186147,0.175228,0.210705,0.180856,0.154286
2,32615,0.159699,0.156954,0.104087,0.204362,0.135775,0.223156,0.164033,0.165670,0.191627,...,0.178529,0.185764,0.145231,0.219777,0.185088,0.134226,0.144906,0.158298,0.126343,0.165897
3,33487,0.281966,0.336183,0.156023,0.152620,0.309022,0.298259,0.381064,0.359054,0.293379,...,0.229931,0.326425,0.355035,0.215321,0.330720,0.332242,0.358194,0.329890,0.295991,0.189020
4,33506,0.302426,0.352408,0.233614,0.284450,0.303648,0.342613,0.329029,0.325627,0.334627,...,0.354339,0.326265,0.332248,0.350640,0.325848,0.327162,0.316171,0.356357,0.243650,0.373440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4370,38129,0.194596,0.140763,0.156023,0.309178,0.062303,0.123920,0.091948,0.065702,0.152442,...,0.138951,0.140398,0.107074,0.231495,0.128613,0.192594,0.172198,0.130486,0.167542,0.136738
4371,37927,0.302082,0.241679,0.108114,0.177148,0.184619,0.189646,0.287789,0.262271,0.199379,...,0.131296,0.187586,0.208741,0.149777,0.215194,0.317767,0.280365,0.281211,0.428499,0.103744
4372,48198,0.042291,0.069988,0.017582,0.062912,0.047179,0.097750,0.043181,0.066829,0.057174,...,0.078290,0.044853,0.075570,0.083728,0.062037,0.079043,0.030040,0.058713,0.031718,0.076137
4373,44495,0.196182,0.148834,0.167027,0.316892,0.067820,0.110687,0.091410,0.062737,0.142936,...,0.137008,0.129574,0.097797,0.236830,0.128847,0.205176,0.171096,0.135547,0.187666,0.135618


find the sim score that has the highest accuracy

In [61]:
df_sim_scores = pd.read_csv('user_sim_scores.csv')
df_true = pd.read_csv('anime_matrix_all.csv')

for 1s 0.2787054322727589

for 0s 0.16166032987583528

In [63]:
row = range(len(df_sim_scores))
col = range(1,df_sim_scores.shape[1])
accuracy_for_all_users = []

for i in tqdm(col):
    # if i == 2: break
    try:
        accuracy_per_user = []
        for j in row:
            if df_true.iat[j,i] == 0:
                try:
                    accuracy_per_user.append(df_sim_scores.iat[j,i])
                except: pass
        # if math.isnan(np.mean(accuracy_per_user)):
        #     pass
        if (len(accuracy_per_user)) > 0:
            accuracy_for_all_users.append(np.mean(accuracy_per_user))
    except: pass

average_sim_score = np.mean(accuracy_for_all_users)
average_sim_score

100%|██████████| 9028/9028 [01:43<00:00, 87.00it/s]


0.16166032987583528

find the best sim score with the highest accuracy

In [30]:
df_true = df_true.drop(columns='anime')
df_true

,ilampan,Zephyyr,Alister-Dark,AMechaDegenarate,Matthew_Myers,eezairi,Neji4460,Aguena_Jun,andrzejszczelba,Ljuder,...,Jaxelar,BitSneakySvenpai,PeelyPeelyBoi,Tray1990,Kilax,Darkidda,chidoriP,snepback,Prism8Azure,urban_orange
0,0,1,0,0,0,1,1,1,1,0,...,1,1,1,1,1,0,1,1,0,0
1,0,1,0,0,0,0,0,1,1,0,...,0,1,1,0,0,1,1,1,0,0
2,0,1,0,1,0,1,1,0,1,0,...,0,0,0,1,0,0,0,1,1,0
3,0,1,0,0,0,0,1,1,0,0,...,0,1,1,0,1,1,1,0,0,0
4,0,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4371,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4372,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4373,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


best percentage seems to be 0.22 at accuracy of 0.04

In [48]:
percentages = [0.2,0.22,0.24,0.26,0.28,0.3,0.32,0.34,0.36,0.38,0.4,0.42,0.44,0.46,0.48,0.5]
accuracy = {}
df_true = pd.read_csv('anime_matrix_full.csv')
df_true = df_true.drop(columns='anime')

for i in tqdm(percentages):
    file_name = 'user_threshold_full_' + str(i) + '.csv'
    df_accuracy = pd.read_csv(file_name)
    df_accuracy = df_accuracy.drop(columns='anime')
    accuracy.update({i:model_config(x=df_accuracy,y=df_true)})
    
accuracy = sorted(accuracy.items(),key=lambda item: item[1])
accuracy

100%|██████████| 16/16 [35:50<00:00, 134.43s/it]


[(0.5, 0.001142857142857143),
 (0.3, 0.014857142857142857),
 (0.4, 0.014857142857142857),
 (0.36, 0.018285714285714287),
 (0.46, 0.02057142857142857),
 (0.24, 0.021714285714285714),
 (0.32, 0.021714285714285714),
 (0.34, 0.021714285714285714),
 (0.26, 0.024),
 (0.28, 0.026285714285714287),
 (0.48, 0.026285714285714287),
 (0.44, 0.02857142857142857),
 (0.2, 0.030857142857142857),
 (0.38, 0.03542857142857143),
 (0.42, 0.038857142857142854),
 (0.22, 0.04)]

In [109]:
from sklearn import preprocessing
import matplotlib.pyplot as plt

new = pd.DataFrame()

df_accuracy = pd.read_csv('top_and_bottom_anime.csv')
df_accuracy = df_accuracy.drop(columns='Id')
df_accuracy = df_accuracy.drop(columns='Rating')

le = preprocessing.LabelEncoder()
for i in range(df_accuracy.shape[1]):
    temp = df_accuracy.iloc[:,i].values.tolist()
    temp = le.fit_transform(temp)
    new[df_accuracy.columns[i]] = temp

df_true = pd.read_csv('anime_matrix_all.csv')
df_true = df_true.drop(columns='anime')
df_true = df_true['ilampan']

K = []
training = []
test = []
scores = {}

x = new
y = df_true
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)



# for k in tqdm(range(1,300)):
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train,y_train)

training_score = knn.score(x_train,y_train)
test_score = knn.score(x_test,y_test)
K.append(k)

training.append(training_score)
test.append(1-test_score)
scores[k] = [training_score, test_score]


fig, ax = plt.subplots()
ax.plot(K, test, 'bx-')
ax.set(xlabel="k",
       ylabel="Error Rate",
       title="Error Rate vs K Values")

lowest_error = min(test)
k_of_lowest = test.index(lowest_error)
print('The lowest error rate is ' + str(lowest_error))
print('The accuracy rate is ' + str(1-lowest_error))
print('The k value that gives the lowest error rate is ' + str(k_of_lowest))

plt.show()

  0%|          | 0/99 [00:37<?, ?it/s]


KeyboardInterrupt: 

decision tree

In [39]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.multioutput import MultiOutputClassifier

new = pd.DataFrame()

df_accuracy = pd.read_csv('top_and_bottom_anime.csv')
df_accuracy = df_accuracy.drop(columns='Id')
df_accuracy = df_accuracy.drop(columns='Rating')


le = preprocessing.LabelEncoder()
for i in range(df_accuracy.shape[1]):
    temp = df_accuracy.iloc[:,i].values.tolist()
    temp = le.fit_transform(temp)
    new[df_accuracy.columns[i]] = temp

df_true = pd.read_csv('anime_matrix_all.csv')
df_true = df_true.drop(columns='anime')
df_true = df_true.iloc[:,:500]

x = new
y = df_true
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

# clf = MultiOutputClassifier(DecisionTreeClassifier())
clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)
# filename = 'decisiontree_model.sav'
# pickle.dump(clf, open(filename, 'wb'))
# loaded_model = pickle.load(open(filename,'rb'))
result = clf.score(x_test,y_test)
result

0.05357142857142857

In [8]:
y_test

,ilampan,Zephyyr,Alister-Dark,AMechaDegenarate,Matthew_Myers,eezairi,Neji4460,Aguena_Jun,andrzejszczelba,Ljuder,...,Jaxelar,BitSneakySvenpai,PeelyPeelyBoi,Tray1990,Kilax,Darkidda,chidoriP,snepback,Prism8Azure,urban_orange
149,0,1,0,1,0,0,1,1,1,1,...,0,1,1,1,1,1,1,1,0,1
194,1,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,1,0,0,0
368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
339,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
440,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,0,1,0,0,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,0,0
18,0,1,0,0,1,1,1,1,1,0,...,0,1,1,0,0,1,1,1,0,0
246,0,1,0,0,0,0,1,0,1,0,...,0,1,0,0,1,1,1,1,0,0
156,1,1,1,1,0,0,0,0,0,0,...,1,0,0,1,1,0,0,1,0,1


item to item model

In [104]:
#caluclate accuracy
df_true = pd.read_csv('anime_matrix_all.csv')

value = 0
prediction_matrix = pd.read_csv('user_threshold_all.csv')
pred = prediction_matrix.values
true = df_true.values
length_row = len(pred)
length_col = len(pred[0])
for i in tqdm(range(length_row)):
    for j in range(1,length_col):
        if pred[i,j] == true[i,j]:
            value += 1

print(value)
print(value / ((length_row) * (length_col - 1)))

100%|██████████| 556/556 [00:01<00:00, 287.21it/s]

3765787
0.750221333788087


random forest

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.multioutput import MultiOutputClassifier

new = pd.DataFrame()

df_accuracy = pd.read_csv('top_and_bottom_anime.csv')
df_accuracy = df_accuracy.drop(columns='Id')
df_accuracy = df_accuracy.drop(columns='Rating')


le = preprocessing.LabelEncoder()
for i in range(df_accuracy.shape[1]):
    temp = df_accuracy.iloc[:,i].values.tolist()
    temp = le.fit_transform(temp)
    new[df_accuracy.columns[i]] = temp

df_true = pd.read_csv('anime_matrix_all.csv')
df_true = df_true.drop(columns='anime')
df_true = df_true

x = new
y = df_true
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

clf = MultiOutputClassifier(RandomForestClassifier())
# clf = RandomForestClassifier()
clf.fit(x_train,y_train)
# filename = 'randomforest_model.sav'
# pickle.dump(clf, open(filename, 'wb'))
result = clf.score(x_test,y_test)
result

0.008928571428571428

In [4]:
y_test

,ilampan,Zephyyr,Alister-Dark,AMechaDegenarate,Matthew_Myers,eezairi,Neji4460,Aguena_Jun,andrzejszczelba,Ljuder,...,Jaxelar,BitSneakySvenpai,PeelyPeelyBoi,Tray1990,Kilax,Darkidda,chidoriP,snepback,Prism8Azure,urban_orange
348,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
407,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
454,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
413,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
189,1,0,0,1,0,0,0,0,1,0,...,1,0,0,1,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
402,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
429,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
